In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
import xgboost
from xgboost import  XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical

import pandas as pd
import numpy as np

tf.get_logger().setLevel('ERROR')
tf.test.is_gpu_available()


2024-03-08 00:31:24.931140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-08 00:31:24.931245: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-08 00:31:24.931264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-08 00:31:24.931420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-08 00:31:24.931428: I tensorflow/core/common_runtime/gpu/gpu

True

In [89]:
apple_data = pd.read_csv("apple_processed_data.csv", index_col=0)
tesla_data = pd.read_csv("tesla_processed_data.csv", index_col=0)

apple_data['month'] = apple_data['month'].astype('category')
apple_data['day'] = apple_data['day'].astype('category')

tesla_data['month'] = tesla_data['month'].astype('category')
tesla_data['day'] = tesla_data['day'].astype('category')

In [90]:
tesla_data.head()

,Date,close,open,lowest,highest,total_vol,mean_vol,std_vol,news,month,day,is_up
0,2015-01-02,14.613,14.865,14.217,14.883,59157390,151685.615385,155760.920466,,1,2,0
1,2015-01-05,14.001,14.351,13.811,14.433,68662800,176058.461538,168290.638752,,1,5,1
2,2015-01-06,14.079,14.003,13.614,14.280,80752635,207058.038462,152662.243747,,1,6,0
3,2015-01-07,14.054,14.219,13.985,14.319,38728110,99558.123393,100907.065350,['BMW revives wireless charging to reduce elec...,1,7,0
4,2015-01-08,14.037,14.216,14.001,14.253,43839960,112699.125964,130930.670726,['How oil at $50 could change almost everythin...,1,8,0


In [91]:
def train_valid_test(data, target, split):
    train, valid, test = split
    n = data.shape[0]
    train_end = int(train * n)
    valid_end = int(n * (train + valid))
    print(valid_end)
    x_train, y_train = data[:train_end], target[:train_end]
    x_val, y_val = data[train_end:valid_end], target[train_end:valid_end]
    x_test, y_test = data[valid_end:], target[valid_end:]

    return x_train, y_train, x_val, y_val, x_test, y_test

In [92]:
data = apple_data
lag = 10
data['ema'] = data['close'].ewm(span=lag).mean()
for i in range(1, lag+1):
    data['lag_' + str(i)] = data['close'].shift(i)

data = data.iloc[lag:]

In [93]:
data.head()

,Date,close,open,lowest,highest,total_vol,mean_vol,std_vol,news,month,...,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20
20,2015-02-02,26.578,26.454,25.980,26.724,220454064,565266.830769,413035.047275,"[""Apple's spectacular Q3 performance from an I...",2,...,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802,23.807,24.490
21,2015-02-03,26.573,26.575,26.322,26.706,174221580,446722.000000,328806.289399,"[""Apple's spectacular Q3 performance from an I...",2,...,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802,23.807
22,2015-02-04,26.786,26.533,26.479,27.025,252725380,648013.794872,712862.564754,Steve Jobs's biopic will hit the screens in Oc...,2,...,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802
23,2015-02-05,26.979,26.938,26.794,27.068,144629728,370845.456410,376791.267099,Cashless ecosystem: National Payment Corporati...,2,...,24.546,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143
24,2015-02-06,26.748,27.020,26.615,27.073,155597272,398967.364103,297534.236592,Global leaders to explore 'technology 20 years...,2,...,25.182,24.546,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066


In [94]:
predictors = data.drop(['is_up', 'Date', 'news', 'day', 'month', 'total_vol', 'mean_vol', 'std_vol', 'open'], axis=1)
target = data['is_up']

In [95]:
x_train, y_train, x_val, y_val, x_test, y_test = train_valid_test(predictors, target, (0.6, 0.2, 0.2))


1762


In [96]:
predictors.head()

,close,lowest,highest,ema,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20
20,26.578,25.980,26.724,25.283226,26.251,26.650,25.839,24.456,25.339,25.317,...,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802,23.807,24.490
21,26.573,26.322,26.706,25.421337,26.578,26.251,26.650,25.839,24.456,25.339,...,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802,23.807
22,26.786,26.479,27.025,25.565756,26.573,26.578,26.251,26.650,25.839,24.456,...,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143,23.802
23,26.979,26.794,27.068,25.713750,26.786,26.573,26.578,26.251,26.650,25.839,...,24.546,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066,24.143
24,26.748,26.615,27.073,25.821038,26.979,26.786,26.573,26.578,26.251,26.650,...,25.182,24.546,24.358,23.746,23.936,24.600,24.697,24.481,25.095,25.066


## Transform data

In [97]:
sc = MinMaxScaler()
sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_val_scaled = sc.transform(x_val)
x_test_scaled = sc.transform(x_test)


X_train_for_compare = np.append(x_train_scaled, x_val_scaled, axis=0)
Y_train_for_compare = np.append(y_train, y_val, axis=0)

## Baseline model
According to EDA, the market was up ~ 52% of the days, so the simplest model would be always predict up, which could potentially achieve 52% accuracy.

In [98]:
counting = data['is_up'].value_counts()
BASELINE = counting[1] / np.sum(counting)
BASELINE

0.5256468452110759

In [99]:
accuracy_score(y_val, [1] * len(y_val))

0.5374149659863946

The baseline model has accuracy of ~ **54%**

## Default hyperparameters with some Algo

Logistic regression

In [100]:
lr = LogisticRegression(random_state = 12)
lr.fit(x_train_scaled, y_train)

print("Training accuracy:", lr.score(x_train_scaled, y_train))
print("Validation accuracy:", lr.score(x_val_scaled, y_val))

Training accuracy: 0.5397426192278577
Validation accuracy: 0.5510204081632653


K Nearest Neighbors:

In [101]:
knn = KNeighborsClassifier()
knn.fit(x_train_scaled, y_train)

print("Training accuracy:", knn.score(x_train_scaled, y_train))
print("Validation accuracy:", knn.score(x_val_scaled, y_val))

Training accuracy: 0.6805450416351249
Validation accuracy: 0.46938775510204084


SVM

In [102]:
svm = SVC()
svm.fit(x_train_scaled, y_train)
print("Training accuracy:", svm.score(x_train_scaled, y_train))
print("Validation accuracy:", svm.score(x_val_scaled, y_val))

Training accuracy: 0.5299015897047691
Validation accuracy: 0.4943310657596372


Naive Bayes

In [103]:
bnb = BernoulliNB()
bnb.fit(x_train_scaled, y_train)
print("Training accuracy:", bnb.score(x_train_scaled, y_train))
print("Validation accuracy:", bnb.score(x_val_scaled, y_val))


Training accuracy: 0.5314155942467828
Validation accuracy: 0.5374149659863946


Random Forest Classifier

In [104]:
rf = RandomForestClassifier(bootstrap=True, random_state = 7)
rf.fit(x_train_scaled, y_train)

print("Training accuracy:", rf.score(x_train_scaled, y_train))
print("Validation accuracy:", rf.score(x_val_scaled, y_val))

Training accuracy: 1.0
Validation accuracy: 0.47165532879818595


In [105]:
sorted(list(zip(predictors.columns, rf.feature_importances_)), key= lambda x: -x[1])

[('lag_7', 0.045138676515350994),
 ('close', 0.04431636153993201),
 ('lag_20', 0.04418274716926696),
 ('lag_8', 0.04415302306653415),
 ('lag_19', 0.04401352202367309),
 ('lag_13', 0.04365725178997353),
 ('lag_16', 0.04336823038092861),
 ('lag_5', 0.04318335663764783),
 ('lowest', 0.04272730951636593),
 ('lag_4', 0.042443121748897764),
 ('lag_12', 0.04214385369069147),
 ('lag_9', 0.04204109360042356),
 ('highest', 0.04192365352242163),
 ('lag_17', 0.04188088727612056),
 ('lag_14', 0.041612483145159224),
 ('lag_6', 0.04094444564718164),
 ('lag_18', 0.04063767872556414),
 ('lag_15', 0.04055890586332204),
 ('lag_10', 0.04010821366419804),
 ('lag_1', 0.04010268308749807),
 ('lag_11', 0.039933679791110795),
 ('lag_2', 0.03834359347079784),
 ('lag_3', 0.03744867033671835),
 ('ema', 0.03513655779022182)]

CatBoostClassifier

In [106]:
cat = CatBoostClassifier(silent=True, random_state = 12)
cat.fit(x_train_scaled, y_train)

print("Training accuracy:", cat.score(x_train_scaled, y_train))
print("Validation accuracy:", cat.score(x_val_scaled, y_val))
print("Test accuracy:", cat.score(x_test_scaled, y_test))

Training accuracy: 0.8569265707797123
Validation accuracy: 0.47165532879818595
Test accuracy: 0.48299319727891155


XGBoost Classifier 

In [107]:
xg = XGBClassifier()
xg.fit(x_train_scaled, y_train)

print("Training accuracy:", xg.score(x_train_scaled, y_train))
print("Validation accuracy:", xg.score(x_val_scaled, y_val))
print("Test accuracy:", xg.score(x_test_scaled, y_test))

Training accuracy: 1.0
Validation accuracy: 0.47619047619047616
Test accuracy: 0.48299319727891155


Simple Deep Learning

In [108]:
x_train_lstm = np.reshape(X_train_for_compare, (X_train_for_compare.shape[0], X_train_for_compare.shape[1], 1))


model = tf.keras.Sequential()
unit = 50
model.add(LSTM(unit, return_sequences=True, input_shape=(x_train_lstm.shape[1], 1)))
model.add(LSTM(unit))
model.add(Dense(units=25))
model.add(Dense(1, activation = 'tanh'))

# optimizer = tf.keras.optimizers.Nadam()
optimizer = tf.keras.optimizers.Adagrad()

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])
hist = model.fit(x_train_lstm, Y_train_for_compare, validation_data = (x_val_scaled, y_val), batch_size=1, epochs=1)

threshold = 0
y_val_pred = (model.predict(x_val_scaled) >= threshold).astype(int)
print("Training accuracy:", accuracy_score(y_train, (model.predict(x_train_scaled) > threshold).astype(int)))
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))
print("Test accuracy:", model.evaluate(x_test_scaled, y_test))

42/42 [==============================] - 0s 3ms/step
Training accuracy: 0.5246025738077215
Validation accuracy: 0.5374149659863946
14/14 [==============================] - 0s 4ms/step - loss: 0.6963 - accuracy: 0.5170
Test accuracy: [0.6962910294532776, 0.5170068144798279]


## Optimize hyper params

In [109]:
import optuna
from optuna import Trial

optuna.logging.set_verbosity(optuna.logging.WARNING)

n_trials = 100
random_state = 7

### Logis Regression

In [110]:
def lr_optimize(trial: Trial):
    solver = trial.suggest_categorical("solver", ["newton-cholesky", "newton-cholesky", "lbfgs"])
    penalty = trial.suggest_categorical("penalty", ['l2', None])
    lr = LogisticRegression(penalty=penalty, solver=solver, random_state=random_state, max_iter=500, multi_class = 'ovr')
    lr.fit(x_train_scaled, y_train)
    y_val_pred = lr.predict(x_val_scaled)
    return accuracy_score(y_val, y_val_pred)

lr_study = optuna.create_study(direction="maximize")
lr_study.optimize(lr_optimize, n_trials=n_trials)

lr_study = optuna.create_study(direction="maximize")
lr_study.optimize(lr_optimize, n_trials=n_trials)

### Random Forest

In [111]:
def rf_optimize(trial: Trial):
    n_estimators = trial.suggest_int('n_estimators', low=10, high = 100)
    criterion = trial.suggest_categorical("criterion", ['gini', 'entropy', 'log_loss'])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, bootstrap=bootstrap)
    lr.fit(x_train_scaled, y_train)
    y_val_pred = lr.predict(x_val_scaled)
    return accuracy_score(y_val, y_val_pred)

rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(rf_optimize, n_trials=n_trials)
print("Best params", rf_study.best_params)
print("Best accuracy", rf_study.best_value)

Best params {'n_estimators': 90, 'criterion': 'entropy', 'bootstrap': True}
Best accuracy 0.5510204081632653


### CatBoost

In [112]:
def catboost_optimize(trial: Trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'loss_function': 'Logloss',
        'verbose': False
    }
    model = CatBoostClassifier(**param)
    model.fit(x_train_scaled, y_train, eval_set=[(x_val_scaled, y_val)], early_stopping_rounds=100, verbose=0)
    preds = model.predict(x_val_scaled)
    accuracy = accuracy_score(y_val, preds)
    return accuracy

cat_study = optuna.create_study(direction="maximize")
cat_study.optimize(catboost_optimize, n_trials=n_trials)
print("Best params", cat_study.best_params)
print("Best accuracy", cat_study.best_value)

Best params {'iterations': 135, 'depth': 5, 'learning_rate': 0.056908794946172625, 'random_strength': 6, 'bagging_temperature': 0.11268454956917995, 'l2_leaf_reg': 6.353060743995894, 'border_count': 224}
Best accuracy 0.5374149659863946


### Neural Network

In [113]:
def create_neural_network_model():
    nn = tf.keras.Sequential()
    nn.add(Dense(len(x_train_scaled[0])))
    nn.add(Dense(1, activation = "sigmoid"))

    return nn

def create_lstm(units):
    model = tf.keras.Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(x_train_lstm.shape[1], 1)))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(units=units))
    model.add(Dense(1, activation = 'sigmoid'))
    return model
    
def create_neural_network_study(trial: Trial):
    model = create_neural_network_model()
    optimizer = trial.suggest_categorical('optimizer', 
                                         ["Adam", "SGD", "RMSprop", "Adagrad"])
    batch_size = trial.suggest_int('batch_size', low=1, high = 15)
    epochs = trial.suggest_int('epochs', low=1, high=5)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    model.compile(optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=lr), loss="binary_crossentropy", metrics=['accuracy'])
    model.fit(x_train_lstm, Y_train_for_compare, validation_data = (x_val_scaled, y_val), batch_size=batch_size, epochs=epochs, verbose=0)
    threshold = 0.5
    y_val_pred = (model.predict(x_val_scaled) >= threshold).astype(int)
    return accuracy_score(y_val, y_val_pred)

def create_lstm_study(trial: Trial):
    units = trial.suggest_int("units", low=10, high = 50)
    model = create_lstm(units)
    optimizer = trial.suggest_categorical('optimizer', 
                                         ["Adam", "SGD", "RMSprop", "Adagrad"])
    batch_size = trial.suggest_int('batch_size', low=1, high = 15)
    epochs = trial.suggest_int('epochs', low=1, high=5)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    model.compile(optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=lr), loss="binary_crossentropy", metrics=['accuracy'])
    model.fit(x_train_lstm, Y_train_for_compare, validation_data = (x_val_scaled, y_val), batch_size=batch_size, epochs=epochs, verbose=0)
    threshold = 0.5
    y_val_pred = (model.predict(x_val_scaled) >= threshold).astype(int)
    return accuracy_score(y_val, y_val_pred)

def run_study(function):
    # Create an Optuna study and optimize the objective
    study = optuna.create_study(direction='maximize')
    study.optimize(function, n_trials=10)
    
    print("Best trial:")
    trial = study.best_trial
    
    print(f"Accuracy: {trial.value}")
    print("Best hyperparameters:")
    for key, value in trial.params.items():
        print(f"{key}: {value}")

In [114]:
run_study(create_lstm_study)

14/14 [==============================] - 0s 3ms/step
Best trial:
Accuracy: 0.5578231292517006
Best hyperparameters:
units: 12
optimizer: Adam
batch_size: 13
epochs: 3
lr: 0.001526401264743937


In [115]:
def create_lstm_2(units):
    model = tf.keras.Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(x_train_lstm.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(units=units // 2))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

def create_lstm_study_2(trial: Trial):
    units = trial.suggest_int("units", low=10, high = 50)
    model = create_lstm_2(units)
    optimizer = trial.suggest_categorical('optimizer', 
                                         ["Adam", "SGD", "RMSprop", "Adagrad"])
    batch_size = trial.suggest_int('batch_size', low=1, high = 15)
    epochs = trial.suggest_int('epochs', low=1, high=5)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    model.compile(optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=lr), loss="binary_crossentropy", metrics=['accuracy'])
    model.fit(x_train_lstm, Y_train_for_compare, validation_data = (x_val_scaled, y_val), batch_size=batch_size, epochs=epochs, verbose=0)
    threshold = 0.5
    y_val_pred = (model.predict(x_val_scaled) >= threshold).astype(int)
    return accuracy_score(y_val, y_val_pred)

run_study(create_lstm_study_2)

14/14 [==============================] - 0s 3ms/step
Best trial:
Accuracy: 0.5374149659863946
Best hyperparameters:
units: 39
optimizer: Adagrad
batch_size: 4
epochs: 2
lr: 0.00035360636710591654


In [116]:
def create_lstm_gru(units):
    model = tf.keras.Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(x_train_lstm.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(GRU(units))
    model.add(Dropout(0.2))
    model.add(Dense(units=units // 2))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

def create_lstm_gru_study(trial: Trial):
    units = trial.suggest_int("units", low=10, high = 50)
    model = create_lstm_2(units)
    optimizer = trial.suggest_categorical('optimizer', 
                                         ["Adam", "SGD", "RMSprop", "Adagrad"])
    batch_size = trial.suggest_int('batch_size', low=1, high = 15)
    epochs = trial.suggest_int('epochs', low=1, high=5)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    model.compile(optimizer=getattr(tf.keras.optimizers, optimizer)(learning_rate=lr), loss="binary_crossentropy", metrics=['accuracy'])
    model.fit(x_train_lstm, Y_train_for_compare, validation_data = (x_val_scaled, y_val), batch_size=batch_size, epochs=epochs, verbose=0)
    threshold = 0.5
    y_val_pred = (model.predict(x_val_scaled) >= threshold).astype(int)
    return accuracy_score(y_val, y_val_pred)

run_study(create_lstm_gru_study)

14/14 [==============================] - 0s 3ms/step
Best trial:
Accuracy: 0.5374149659863946
Best hyperparameters:
units: 21
optimizer: Adam
batch_size: 15
epochs: 4
lr: 0.018368809250582617
